```.py```で挑戦した証。結局できないという結果は変わらなかった。

In [ ]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random, math, warnings
from sklearn.feature_selection import SelectKBest, f_regression
#from sklearn.feature_selection import SelectKBest, chi2
import pprint
import itertools

num_select = 3

every_MCS = 100

# 降順である{1/b}の作成
# 最高温度（スタート地点）
Max_temper = 2000
b = [100/j for j in reversed(range(1, Max_temper+1))]

# 説明変数（こっちの特徴量上位5つ取り出す）
X = pd.read_table('geneX_df.txt', sep='\t')

# 目的変数
y = pd.read_table('ERY_df.txt', sep='\t')

# Seriesにした
ori_y = y.loc[:, 'ER_Status_nature2012']

# サンプル数
num_sample = X.shape[0]
# 特徴量の数
num_feature = X.shape[1]

for i in range(len(y)):
    if y.iloc[i, 0] == "Positive":
        y.iloc[i, 0] = np.float(1)
    else:
        y.iloc[i, 0] = np.float(0)

# サンプル数
num_sample = X.shape[0]

# 特徴量の数
num_feature = X.shape[1]

# K = len(y_tuple_list)/Max_temper #419.4304
K = 2*210

ori_y = y

# mask_oriの計算（以降のエネルギー計算の基準となる）
# ori_y == yとなる場合だけ、E(y)==0となる仕様

selector = SelectKBest(score_func=f_regression, k=num_select)
selector.fit(X, ori_y)
mask_ori = selector.get_support()


def E(y):
    selector = SelectKBest(score_func=f_regression, k=num_select)
    selector.fit(X, y)
    mask_each = selector.get_support()
    
    energy_trait_sum = 0
    for i in range(len(mask_each)):
        if mask_each[i] != mask_ori[i]:
            energy_trait_sum += 1
    return energy_trait_sum

# これからi(温度)ごとに入れていく。一行に入っているのもlistにしたい(行i, 列k, 要素Ene)
Ene = []

# これからi(温度)ごとに、選択したyを記録していく。(行i, 要素num_tl)
y_list = []

# Wが大きい（Eneが小さい）の方々のメンバー(上位メンバー)所属(k)が入ったlist
high_P_Klist = []
ll = len(list(itertools.product('10', repeat=num_sample)))-1

for i in range(Max_temper):   
    Ene.append({})
    y_list.append([])
    for k in range(K):
        tl = list(itertools.product('10', repeat=num_sample))[random.randint(0, ll)]
        Ene[-1][k]= E(pd.Series(tl))
        y_list[-1].append(tl)

        r = math.exp(-Ene[i][k]*b[i]-(-Ene[i-1][k]*b[i-1]))
        R = np.random.uniform(0.0, 1.0)
        # 乱数R > エネルギー差の大きさを用いた計算
        if R > r:
            Ene[-1][k] = Ene[i-1][k]
            y_list[-1][k] = y_list[i-1][k]
    
    if i % every_MCS == 0:
        Ei_sorted = sorted(Ene[-1].items(), key=lambda x:x[1])
        tuple_kE_list = Ei_sorted[0:int(len(Ei_sorted)/2)]

        # 上位の方々のメンバー所属(k)が入ったlistを作る
        high_P_Klist.append([])
        for k in range(len(tuple_kE_list)):
            high_P_Klist[-1].append(tuple_kE_list[k][0])

        # 下位メンバーは上位メンバーに変えてしまう
        for k in range(int(K/2)):
            if not k in high_P_Klist[-1]:
                y_list[-1][k] = high_P_Klist[-1][k]
                Ene[-1][k] = Ene[-1][high_P_Klist[-1][k]]

Ene_v_sum_list = []
Ene_v_sum_list_append = Ene_v_sum_list.append
for each_Ene in Ene:
    Ene_v_sum = 0
    for Ene_v in each_Ene.values():
        Ene_v_sum += Ene_v
    Ene_v_sum_list_append(Ene_v_sum/K)
# Ene_v_sum_list
plt.plot(Ene_v_sum_list)
plt.show()
# print(Ene_v_sum_list[-1])

# i(温度)ごとに、選択したyを記録していく。(行i, 要素num_tl)
y_list_df = pd.DataFrame(y_list)